In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import os
os.chdir('/content/drive/MyDrive/DataCollection/tensorflow-ml-nlp-tf2/4.TEXT_CLASSIFICATION/')

In [37]:
os.getcwd()

'/content/drive/MyDrive/DataCollection/tensorflow-ml-nlp-tf2/4.TEXT_CLASSIFICATION'

## 4.1.3 Linear Regression Example with Word2Vec

### Word2Vec Feature Example

In [38]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [39]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [40]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA) # 전처리된 data를 사용

In [41]:
list(train_data['review'])[:2] ## series data를 바로 list로 바꿀 수 있다.

['stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate worki

In [42]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [43]:
reviews[0].split()[:3]

['stuff', 'going', 'moment']

In [44]:
sentences = []
for review in reviews:
    sentences.append(review.split())
    ## split 함수안에 아무것도 안적혀있으면 띄어쓰기 기준으로 split.. 따라서 split() == split(' ')
    ## word2vec을 사용하기 위해서 각 단어로 구분된 리스트로 바꾼다.

In [45]:
sentences[0][:3]

['stuff', 'going', 'moment']

In [46]:
num_features = 300   # num. of embedding dimensions.
min_word_count = 40  # 모델에 의미 있는 단어를 가지고 학습하기 위해 40 미만의 빈도 수의 단어들은 학습 x
num_workers = 4      # CPU 쿼드코어를 이용 (사용 프로세스의 갯수)
context = 10         # window size. 앞뒤로 10개를 보겠다는 의미
downsampling = 1e-3  # 빠른 학습을 위해 정답 단어 라벨에 대한 다운샘플링 비율을 지정. 보통 0.001을 사용
##(추가 의견: downsampling 수치가 뭔지에 대해서 약간 찾아봤는데, 전체 단어의 0.001을 어떤 단어가 넘는 순간, 
##빈번하게 등장한다고 판단하고 이 단어에 대해 다운샘플링을 진행하는 것으로 보임)
##출처: https://dalpo0814.tistory.com/13 [deeep]

In [47]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [ ]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

In [49]:
# 모델 저장하기
model_name = "300features_40minwords_10context"
model.save(model_name)

2020-12-06 12:26:44,432 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2020-12-06 12:26:44,434 : INFO : not storing attribute vectors_norm
2020-12-06 12:26:44,435 : INFO : not storing attribute cum_table
2020-12-06 12:26:44,733 : INFO : saved 300features_40minwords_10context


In [50]:
def get_features(words, model, num_features):  ## 일반적인 look up table을 만드는 것과 동일하다.
    feature_vector = np.zeros((num_features),dtype=np.float32) # 하나의 벡터를 만드는 과정에서 속도를 빠르게 하기 위해 np.zeros를 미리 만듦!

    num_words = 0
    index2word_set = set(model.wv.index2word)  # 단어 vocab

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector
    ## 여기까지하면 리뷰 한 줄에 대한 vectorized representation 만들어진다.

In [58]:
feature_vector = np.zeros((num_features),dtype=np.float32)
feature_vector.shape

(300,)

In [56]:
model['stuff'].shape  ### model[w]는 각 단어의 vector라는 것을 알기 위해 해봄.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

In [61]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [62]:
test_data_vecs = get_dataset(sentences, model, num_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


이하로는 이전 4.1.3(4)의 tf-idf와 동일하다.

In [63]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [64]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
predicted = lgs.predict(X_test)
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

------------
Accuracy: 0.865600
Precision: 0.858641
Recall: 0.877729
F1-Score: 0.868080
AUC: 0.934383


In [66]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

In [67]:
test_data.head(5)

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


In [68]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [70]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [71]:
test_predicted = lgs.predict(test_data_vecs)

In [72]:
ids = list(test_data['id'])

answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})

In [ ]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)